In [23]:
import json
import pandas as pd
import firebase_admin
from datetime import datetime, timedelta
from firebase_admin import credentials
from firebase_admin import firestore

In [24]:

df = pd.read_csv("../Text Normalization/Tweets.csv")

In [25]:
#df.dropna(inplace= True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4791 entries, 0 to 4790
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  4791 non-null   int64 
 1   Tweet       4791 non-null   object
 2   Location    4791 non-null   object
 3   year        4791 non-null   int64 
 4   month       4791 non-null   int64 
 5   day         4791 non-null   int64 
 6   Sentiment   4791 non-null   object
dtypes: int64(4), object(3)
memory usage: 262.1+ KB


In [ ]:
CredNew = credentials.Certificate('./smartcc-960f7-firebase-adminsdk-ijo0i-27aa89d85e.json')
new = firebase_admin.initialize_app(CredNew,name="new")

db = firestore.client(new)

In [ ]:
print(df.shape[0])

4791


In [82]:
def getPastDateAndTime(daysBefore):
    currentDate = str(datetime.now() - timedelta(days = daysBefore)) 
    splitDateList = currentDate.split(" ")
    dateOnly = splitDateList[0]
    timeOnly = splitDateList[1].split(".")[0]
    dateWithTime = dateOnly + " " + timeOnly

    return [dateOnly, dateWithTime]




def uploadDataToFirebase (collectionName, searchKeyword, df, dataLimit):
    dfLength = df.shape[0]
    iterationCount = 0
    minLimit = 0
    maxLimit = None
    
    if (dfLength > dataLimit):
        iterationCount = int(dfLength / dataLimit) + 1

    for i in range(0, iterationCount):

        if (iterationCount == 1):
            firebaseData = json.loads(json.dumps(df[0:maxLimit].to_dict()))
        else:
            if (i == (iterationCount - 1)):
                maxLimit = None
            else:
                maxLimit = dataLimit * (i + 1)
                
            minLimit = dataLimit * i 

        firebaseData = json.loads(json.dumps(df[minLimit:maxLimit].to_dict()))
        currentDate, currentDateAndTime = getPastDateAndTime(1)
        
        db.collection(collectionName).document(searchKeyword).collection(currentDate).document(currentDateAndTime).set(firebaseData)

    

In [85]:
uploadDataToFirebase("test3", "فودافون", df, 2500)